<a href="https://colab.research.google.com/github/Hillascher5/nlp-tweets-sentiment-analysis/blob/main/Bertweet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Try a specific model for the task

In [1]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import torch
import optuna
import wandb
wandb.login()
# API key - 0cbd7fe3cffd71df993b30edb4fa0db94f114413

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification_split_to_test_maxl_256_bertweet"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1
Mounted at /content/drive


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hillas (hillas-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

In [4]:
# Merge and shuffle for better stratified splits
df_full = pd.concat([df_train, df_test], ignore_index=True)
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

In [5]:
# Try without pre-processing
is_preprocessed = "no_preprocess"
df_full["clean_text"] = df_full["OriginalTweet"]

In [6]:
# Choose pretrained models
bert_model_name = "vinai/bertweet-base"

sentiment_labels = df_full['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_full["Sentiment"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df_full["label"] = df_full["Sentiment"].map(label2id)

In [8]:
# Stratified split: 70% train, 15% val, 15% test
train_val_df, test_df = train_test_split(df_full, test_size=0.15, stratify=df_full["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, stratify=train_val_df["label"], random_state=42)

# Confirm sizes
print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

Train size: 31466
Val size: 6745
Test size: 6744


In [9]:
train_subset_df, _ = train_test_split(
    train_df[["clean_text", "label"]],
    train_size=2000,
    stratify=train_df["label"],
    random_state=42
)

val_subset_df, _ = train_test_split(
    val_df[["clean_text", "label"]],
    train_size=500,
    stratify=val_df["label"],
    random_state=42
)

In [10]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=128)

In [11]:
# Convert DataFrame to Hugging Face Dataset
hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

hf_train = Dataset.from_pandas(train_df[["clean_text", "label"]])
hf_val = Dataset.from_pandas(val_df[["clean_text", "label"]])
hf_test = Dataset.from_pandas(test_df[["clean_text", "label"]])

In [12]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_test = hf_test.map(tokenize_function_bert, batched=True)
tokenized_bert_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [14]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

In [15]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-{is_preprocessed}"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        label_smoothing_factor=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [16]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="vinai/bertweet-base",
        trial=trial,
        run_prefix="bertweet",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

In [17]:
# !huggingface-cli login

In [18]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name=f"bertweet_study_stratify_{is_preprocessed}",
                                 storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bertweet_study_stratify_maxl_256_{is_preprocessed}.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)
wandb.finish()

[I 2025-08-07 20:21:18,350] Using an existing study with name 'bertweet_study_stratify_no_preprocess' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5527, 'grad_norm': 4.736123561859131, 'learning_rate': 8.558019834262654e-06, 'epoch': 1.0}
{'eval_loss': 1.4529064893722534, 'eval_accuracy': 0.37, 'eval_f1_macro': 0.2626007933374692, 'eval_runtime': 1.4985, 'eval_samples_per_second': 333.67, 'eval_steps_per_second': 42.042, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.3716, 'grad_norm': 19.645160675048828, 'learning_rate': 5.720540554993145e-06, 'epoch': 2.0}
{'eval_loss': 1.3430989980697632, 'eval_accuracy': 0.43, 'eval_f1_macro': 0.439498694203324, 'eval_runtime': 1.4634, 'eval_samples_per_second': 341.678, 'eval_steps_per_second': 43.051, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.0791, 'grad_norm': 4.396622180938721, 'learning_rate': 5.403704405701043e-08, 'epoch': 3.0}
{'eval_loss': 1.2245572805404663, 'eval_accuracy': 0.518, 'eval_f1_macro': 0.5271964168414943, 'eval_runtime': 1.4778, 'eval_samples_per_second': 338.336, 'eval_steps_per_second': 42.63, 'epoch': 3.0}
{'train_runtime': 72.449, 'train_samples_per_second': 82.817, 'train_steps_per_second': 10.352, 'train_loss': 1.289187296549479, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-08-07 20:28:13,423] Trial 12 finished with value: 0.5271964168414943 and parameters: {'learning_rate': 2.0263891521378913e-05, 'batch_size': 8, 'num_train_epochs': 3}. Best is trial 11 with value: 0.6351333750181567.


{'eval_loss': 1.2245572805404663, 'eval_accuracy': 0.518, 'eval_f1_macro': 0.5271964168414943, 'eval_runtime': 3.1984, 'eval_samples_per_second': 156.326, 'eval_steps_per_second': 19.697, 'epoch': 3.0}


eval/accuracy,▁▃▄▄▄▂▂▅▅▂▄▅▅▃▆▇███▂▅▅▅
eval/f1_macro,▁▄▅▅▅▃▃▆▆▃▅▆▆▃▇▇███▃▆▆▆
eval/loss,█▅▄▄▄█▅▃▃█▄▃▃▄▁▂▁▄▄▆▃▂▂
eval/runtime,▄▄▄▄▄▁▁▁▁▄▄▄█▄▄▄▄▄▄▄▄▄█
eval/samples_per_second,▂▂▂▂▂███▇▂▂▂▁▂▂▂▂▂▂▂▂▂▁
eval/steps_per_second,█████▅▅▅▄███▁█████████▁
train/epoch,▁▁▃▃▅▅▆▆▆▁▁▃▃▅▅▅▁▁▃▃▅▅▅▁▁▃▃▅▆▆███▁▁▃▃▅▅▅
train/global_step,▂▂▄▄▅▅▇▇▇▁▁▁▁▂▂▂▂▂▄▄▅▅▅▂▂▄▄▅▇▇███▂▂▄▄▅▅▅
train/grad_norm,▂▅▃▃▁▂▂▁▃▂▂▅▃█▁▂▄▂
train/learning_rate,▃▂▂▁▇▄▁▃▂▁█▆▅▃▁▄▃▁
train/loss,█▇▆▅█▇▅█▇▅▇▅▄▂▁█▆▄


In [19]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)

Bert best trial on subset:
{'learning_rate': 3.907368486667445e-05, 'batch_size': 8, 'num_train_epochs': 5}


In [20]:
best_params_bert = best_trial_bert.params
run_name_bert = f"bertweet_final_stratify_{is_preprocessed}-ep{best_params_bert['num_train_epochs']}-lr{best_params_bert['learning_rate']:.1e}-bs{best_params_bert['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test_maxl_256_bertweet", name=run_name_bert, reinit=True)

final_trainer_bert = build_trainer(
    model_checkpoint="vinai/bertweet-base",
    trial=best_trial_bert,
    run_prefix=f"bertweet_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()
final_trainer_bert.evaluate(tokenized_bert_test)
wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/tmp/ipython-input-523513336.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.0036, 'grad_norm': 17.193601608276367, 'learning_rate': 3.127285311927076e-05, 'epoch': 1.0}
{'eval_loss': 0.891286313533783, 'eval_accuracy': 0.7755374351371386, 'eval_f1_macro': 0.7867367440949316, 'eval_runtime': 19.5579, 'eval_samples_per_second': 344.873, 'eval_steps_per_second': 43.154, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.75, 'grad_norm': 0.1666225641965866, 'learning_rate': 2.3464075528477814e-05, 'epoch': 2.0}
{'eval_loss': 0.8648982048034668, 'eval_accuracy': 0.7915492957746478, 'eval_f1_macro': 0.7998869147246579, 'eval_runtime': 19.6388, 'eval_samples_per_second': 343.452, 'eval_steps_per_second': 42.976, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.4997, 'grad_norm': 0.07513552159070969, 'learning_rate': 2.7810451862401745e-08, 'epoch': 5.0}
{'eval_loss': 0.7617508172988892, 'eval_accuracy': 0.8668643439584878, 'eval_f1_macro': 0.8713936014635906, 'eval_runtime': 19.5733, 'eval_samples_per_second': 344.602, 'eval_steps_per_second': 43.12, 'epoch': 5.0}
{'train_runtime': 1685.8542, 'train_samples_per_second': 93.324, 'train_steps_per_second': 11.668, 'train_loss': 0.6928059254555636, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.769802451133728, 'eval_accuracy': 0.8640272835112692, 'eval_f1_macro': 0.8680775747664612, 'eval_runtime': 19.6094, 'eval_samples_per_second': 343.917, 'eval_steps_per_second': 42.99, 'epoch': 5.0}


eval/accuracy,▁▂▆███
eval/f1_macro,▁▂▆███
eval/loss,█▇▃▁▁▂
eval/runtime,▁█▇▁▂▅
eval/samples_per_second,█▁▂█▇▃
eval/steps_per_second,█▁▂▇▇▂
train/epoch,▁▁▃▃▅▅▆▆████
train/global_step,▁▁▃▃▅▅▆▆████
train/grad_norm,▂▁▁█▁
train/learning_rate,█▆▅▃▁
train/loss,█▄▃▂▁


In [21]:
final_trainer_bert.save_model(f"models/w_test_split/bertweet_final_stratify_{is_preprocessed}")
bert_tokenizer.save_pretrained(f"models/w_test_split/bertweet_final_stratify_{is_preprocessed}")
!cp -r models/w_test_split/bertweet_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bertweet_best_model_stratify_maxl_256_{is_preprocessed}"